In [10]:
# Importing libraries
import cv2
import numpy as np

In [12]:
# Path to input video file
path = r"C:\Users\lenovo\OneDrive\Desktop\Optical Light.mp4"
# Load the video or capture from the camera
cap = cv2.VideoCapture(path)

# Set desired frame width and height
frame_width = 640  # Resize width
frame_height = 480  # Resize height

# Brightness factor (1.0 means no change, >1.0 increases brightness)
brightness_factor = 1.5

# Define the codec and create VideoWriter object to save the output video
output_path = r"C:\Users\lenovo\OneDrive\Desktop\Optical_Farneback_Output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 video
out = cv2.VideoWriter(output_path, fourcc, 30.0, (frame_width, frame_height * 2))

# Read the first frame
ret, first_frame = cap.read()

# Resize the first frame
first_frame = cv2.resize(first_frame, (frame_width, frame_height))

# Convert the first frame to grayscale
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (frame_width, frame_height))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the flow
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Set the hue according to the optical flow direction
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255  # Set saturation to maximum
    hsv[..., 0] = angle * 180 / np.pi / 2  # Direction of the flow (Hue)

    # Normalize the magnitude to [0, 255] and apply the brightness factor
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX) * brightness_factor
    hsv[..., 2] = np.clip(hsv[..., 2], 0, 255)  # Ensure values remain within [0, 255]

    # Convert HSV image to BGR to display
    rgb_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Stack the input frame (upper) and optical flow output (lower) vertically
    combined_frame = np.vstack((frame, rgb_flow))

    # Write the combined frame to the output video file
    out.write(combined_frame)

    # Display the combined result
    cv2.imshow('Optical Flow - Farneback', combined_frame)

    # Update previous frame
    prev_gray = gray

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects, and close all windows
cap.release()
out.release()
cv2.destroyAllWindows()